# Modeling a linear regression for predicting counts

# Introducción al machine learning con Scikit-Learn

In [1]:
from sklearn import model_selection, linear_model, metrics, pipeline, preprocessing
import pandas as pd
import numpy as np
import calendar
from datetime import datetime

## Lectura de datos

Vamos a empezar por leer los datos originales. Vamos a hacer un modelo simple para que no necesitemos todos los datos limpios que creamos anteriormente.

In [2]:
df = pd.read_csv("../data/bikes.csv")

In [3]:
df.dtypes

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object

In [4]:
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


Vamos a crear dos variables simples que usaremos en el modelado.

In [5]:
df.datetime = df.datetime.apply(pd.to_datetime)
df['month'] = df.datetime.apply(lambda x : x.month)
df['hour'] = df.datetime.apply(lambda x : x.hour)

In [6]:
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,month,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,1,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,1,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,1,4


In [7]:
df.dtypes

datetime      datetime64[ns]
season                 int64
holiday                int64
workingday             int64
weather                int64
temp                 float64
atemp                float64
humidity               int64
windspeed            float64
casual                 int64
registered             int64
count                  int64
month                  int64
hour                   int64
dtype: object

## Dividiendo nuestros datos

El valor práctico de los modelos proviene de hacer predicciones sobre nuevos datos, por lo que debemos medir el rendimiento en datos que no se usaron para construir el modelo. La forma más sencilla de hacer esto es excluir algunos datos del proceso de creación del modelo y luego usarlos para probar la precisión del modelo en los datos que no se han visto antes. Estos datos se llaman datos de validación o prueba.

Hay una función en `scikit-learn` llamada` train_test_split` que podemos usar para crear una mejor submuestra o los conjuntos de datos y pasar más argumentos para seleccionar diferentes cosas, pero por ahora haremos una muy simple dividir solo por filas.

In [8]:
train_data = df.iloc[:-1000, :]
test_data = df.iloc[-1000:, :]
print(df.shape, train_data.shape, test_data.shape)

(10886, 14) (9886, 14) (1000, 14)


Necesitamos seleccionar la variable objetivo que queremos predecir. Guardaremos esto en una nueva variable llamada train_labels. Esto a menudo se llama `y` también. Haremos lo mismo para los datos de prueba.

Elegiremos predecir los conteos.

Y debido a algunos de los análisis que hicimos antes, no queremos usar algunas de las variables para predecir. También debemos eliminar del tren y probar los datos de la columna que queremos predecir.

In [9]:
train_labels = train_data['count'].values
train_data = train_data.drop(['datetime', 'count', 'casual', 'registered'], axis = 1)
test_labels = test_data['count'].values
test_data = test_data.drop(['datetime', 'count', 'casual', 'registered'], axis = 1)

## Manipulación de datos para el modelado (y tuberías)

También necesitamos modificar los datos que tenemos para que funcionen los modelos integrados en la biblioteca. Para eso primero necesitamos encontrar los diferentes tipos de variables que tenemos. Básicamente tenemos variables binarias, categóricas y numéricas.

In [10]:
binary_data_columns = ['holiday', 'workingday']
binary_data_indices = np.array([(column in binary_data_columns) for column in train_data.columns], dtype = bool)

categorical_data_columns = ['season', 'weather', 'month'] 
categorical_data_indices = np.array([(column in categorical_data_columns) for column in train_data.columns], dtype = bool)

numeric_data_columns = ['temp', 'atemp', 'humidity', 'windspeed', 'hour']
numeric_data_indices = np.array([(column in numeric_data_columns) for column in train_data.columns], dtype = bool)

Lo que construimos aquí fue una lista con una variable booleana que nos dice si el valor en un índice específico corresponde al tipo de variable. Por ejemplo:

In [11]:
categorical_data_indices

array([ True, False, False,  True, False, False, False, False,  True,
       False])

Ahora necesitamos preprocesar los datos que entrarán en el modelo. Para todas las variables, usaremos el `FunctionTransformer` solo para prepararlos para algún cambio y poder unirlos en un Pipe.

El concepto de una tubería es muy importante en el mundo de Python. El propósito de la tubería es ensamblar varios pasos que se pueden validar de forma conjunta al establecer diferentes parámetros.

Quizás tu preprocesamiento requiera solo una de estas transformaciones, como alguna forma de escalado. Pero tal vez necesites unir varias transformaciones y, finalmente, terminar con un estimador de algún tipo. Aquí es donde los Pipelines de Scikit-learn puede ser útiles.

Aquí toma un estimador como cualquier objeto que aprende de los datos; puede ser un algoritmo de clasificación, regresión o agrupamiento o un transformador que extrae / filtra características útiles de datos sin procesar.

In [12]:
help(preprocessing.FunctionTransformer)

Help on class FunctionTransformer in module sklearn.preprocessing._function_transformer:

class FunctionTransformer(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin)
 |  Constructs a transformer from an arbitrary callable.
 |  
 |  A FunctionTransformer forwards its X (and optionally y) arguments to a
 |  user-defined function or function object and returns the result of this
 |  function. This is useful for stateless transformations such as taking the
 |  log of frequencies, doing custom scaling, etc.
 |  
 |  A FunctionTransformer will not do any checks on its function's output.
 |  
 |  Note: If a lambda is used as the function, then the resulting
 |  transformer will not be pickleable.
 |  
 |  .. versionadded:: 0.17
 |  
 |  Read more in the :ref:`User Guide <function_transformer>`.
 |  
 |  Parameters
 |  ----------
 |  func : callable, optional default=None
 |      The callable to use for the transformation. This will be passed
 |      the same arguments as transform, w

Después de eso, escalaremos los datos numéricos y utilizaremos OneHotEncoder para los datos categóricos.

In [13]:
transformer_list = [        
            #binary
            ('binary_variables_processing', preprocessing.FunctionTransformer(lambda data: data[:, binary_data_indices])), 
                    
            #numeric
            ('numeric_variables_processing', pipeline.Pipeline(steps = [
                ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, numeric_data_indices])),
                ('scaling', preprocessing.StandardScaler(with_mean = 0))            
                        ])),
        
            #categorical
            ('categorical_variables_processing', pipeline.Pipeline(steps = [
                ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, categorical_data_indices])),
                ('hot_encoding', preprocessing.OneHotEncoder(handle_unknown = 'ignore'))            
                        ])),
        ]

## Prediciendo con un modelo

Ahora vamos a ajustar una regresión lineal simple.

In [ ]:
help(linear_model.LinearRegression)

In [14]:
regressor = linear_model.LinearRegression()

La tubería primero preprocesa los datos y luego ajusta el modelo LR.

In [15]:
pipe = pipeline.Pipeline(steps = [       
    ('feature_processing', pipeline.FeatureUnion(transformer_list=transformer_list)),
    ('model_fitting', regressor)
    ]
)

In [16]:
pipe

Pipeline(memory=None,
     steps=[('feature_processing', FeatureUnion(n_jobs=1,
       transformer_list=[('binary_variables_processing', FunctionTransformer(accept_sparse=False,
          func=<function <lambda> at 0x1142fb7b8>, inv_kw_args=None,
          inverse_func=None, kw_args=None, pass_y='deprecated',
          valida...)), ('model_fitting', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [17]:
pipe.fit(train_data, train_labels)
predicted = pipe.predict(test_data)

Solo para verificar podemos usar el error Mean Absolute para ver qué tan bien lo hicimos con este modelo simple. Aquí puede leer más sobre esto: https://en.wikipedia.org/wiki/Mean_absolute_error

In [ ]:
help(metrics.mean_absolute_error)

In [18]:
print("MAE: ",  metrics.mean_absolute_error(test_labels, predicted))

MAE:  121.40638334175793
